Before you begin, execute this cell to import numpy and packages from the D-Wave Ocean suite, and all necessary functions for the gate-model framework you are going to use, whether that is the Forest SDK or Qiskit. In the case of Forest SDK, it also starts the qvm and quilc servers.

In [ ]:
%run -i "assignment_helper.py"
%matplotlib inline

# Quantum approximate optimization algorithm

QAOA is a shallow-circuit variational algorithm that is easy to understand if you already grasped quantum annealing. It is, in fact, just a particular type of a larger family of algorithms called variational quantum eigensolvers. Approximating the adiabatic pathway as QAOA does is just one option of how to find the eigenvalues of a system.

Even then, QAOA has many moving elements. Let us import some handy packages and define some functions that we are going to use:

In [ ]:
import itertools
from functools import partial, reduce
from scipy.optimize import minimize
np.set_printoptions(precision=3, suppress=True)

# Functions useful if you're using Qiskit
def pauli_x(qubit, coeff):
    eye = np.eye((n_qubits))
    return Operator([[coeff, Pauli(np.zeros(n_qubits), eye[qubit])]])

def pauli_z(qubit, coeff):
    eye = np.eye((n_qubits))
    return Operator([[coeff, Pauli(eye[qubit], np.zeros(n_qubits))]])

def product_pauli_z(q1, q2, coeff):
    eye = np.eye((n_qubits))
    return Operator([[coeff, Pauli(eye[q1], np.zeros(n_qubits)) * Pauli(eye[q2], np.zeros(n_qubits))]])

**Exercise 1** (2 points). Define a mixing Hamiltonian on two qubits. Store it in an object called `Hm`. If you're doing the assignement in PyQuil, remember that the Hamiltonian should be a list of PauliTerms (and not a PauliSum) in order to be exponentiated easily.

In [ ]:
n_qubits = 2
### BEGIN SOLUTION
Hm = reduce(lambda x, y: x+y,
            [pauli_x(i, 1) for i in range(n_qubits)])
# Forest version
# Hm = [PauliTerm("X", i, 1.0) for i in range(n_qubits)]
### END SOLUTION

In [ ]:
if isinstance(Hm, Operator):
    Hm.to_matrix()
    assert np.alltrue(Hm.matrix.todense() == np.array([[0., 1., 1., 0.],
                                                       [1., 0., 0., 1.],
                                                       [1., 0., 0., 1.],
                                                       [0., 1., 1., 0.]]))
elif isinstance(Hm, list):
    assert len(Hm) == n_qubits
    assert all([isinstance(Hm[i], PauliTerm) for i in range(n_qubits)])
    assert all([Hm[i].compact_str() == '(1+0j)*X{}'.format(i) for i in range(n_qubits)])
else:
    raise ValueError("Unknown type for Hamiltonian!")

**Exercise 2** (2 points). Define the cost Hamiltonian $H_c = -\sigma^Z_1\sigma^Z_2-0.5\sigma^Z_1$.

In [ ]:
### BEGIN SOLUTION
Hc = product_pauli_z(0,1, -1)
Hc += pauli_z(0, -0.5)
# Forest version
# Hc = [PauliTerm("Z", 0, -1.0) * PauliTerm("Z", 1, 1.0), PauliTerm("Z", 0, -0.5)]
### END SOLUTION

In [ ]:
if isinstance(Hc, Operator):
    Hc.to_matrix()
    assert np.alltrue(Hc.matrix == np.array([-1.5, 1.5, 0.5, -0.5]))
    
elif isinstance(Hc, list):
    assert len(Hc) == 2
    assert all([isinstance(Hc[i], PauliTerm) for i in range(n_qubits)])
    assert Hc[0].compact_str() == '(-1+0j)*Z0Z1'
    assert Hc[1].compact_str() == '(-0.5+0j)*Z0'
else:
    raise ValueError("Unknown type for Hamiltonian!")    

**Exercise 3** (2 points). We know that the ground state of the mixer Hamiltonian is the uniform superposition. Create a circuit `circuit_init` that will contain this initial state.

In [ ]:
### BEGIN SOLUTION
init_state_vect = [1 for i in range(2**n_qubits)]
init_state = Custom(n_qubits, state_vector=init_state_vect)
qr = QuantumRegister(n_qubits)
circuit_init = init_state.construct_circuit('circuit', qr)
# Forest version
# circuit_init = Program()
# for i in range(n_qubits):
#     circuit_init += H(i)
### END SOLUTION

In [ ]:
amplitudes = get_amplitudes(circuit_init)
assert np.allclose(amplitudes, np.array([0.5, 0.5, 0.5, 0.5]))

We set $p=2$ and initialize the $\beta_i$ and $\gamma_i$ parameters. 

In [ ]:
p = 2
beta = np.random.uniform(0, np.pi*2, p)
gamma = np.random.uniform(0, np.pi*2, p)

The next step is to create the complete variational circuit, made of $e^{-\beta H}$ and $e^{-\gamma H}$. We will use a function `create_circuit` that takes `gamma` and `beta` as argument, and the state preparation circuit.

In [ ]:
def evolve(hamiltonian, angle, quantum_registers):
    return hamiltonian.evolve(None, angle, 'circuit', 1,
                              quantum_registers=quantum_registers,
                              expansion_mode='suzuki',
                              expansion_order=3)

def create_circuit(circuit_init, beta, gamma):
    if isinstance(circuit_init, qiskit.circuit.quantumcircuit.QuantumCircuit):
        qr = circuit_init.qregs[0]
        circuit_evolv = reduce(lambda x,y: x+y, [evolve(Hc, beta[i], qr) + evolve(Hm, gamma[i], qr)
                                                 for i in range(p)])
        circuit = circuit_init + circuit_evolv
    elif isinstance(circuit_init, pyquil.quil.Program):
        exp_Hm = []
        exp_Hc = []
        for term in Hm:
            exp_Hm.append(exponential_map(term))
        for term in Hc:
            exp_Hc.append(exponential_map(term))
        circuit = Program()
        circuit += circuit_init
        for i in range(p):
            for term_exp_Hm in exp_Hm:
                circuit += term_exp_Hm(-beta[i])
            for term_exp_Hc in exp_Hc:
                circuit += term_exp_Hc(-gamma[i])
    return circuit

Finally, we need a function `evaluate_circuit` to compute the average energy of the circuit, i.e. compute $\langle\psi(\beta, \gamma)|H_c|\psi(\beta, \gamma)\rangle$ where $|\psi(\beta, \gamma)\rangle$ is the circuit built above. This function should take a unique argument `beta_gamma` (concatenation of the lists `beta` and `gamma`) in order to be used directly by optimizers, and return a real value corresponding to the expectancy of $H_c$.

In [ ]:
def evaluate_circuit(beta_gamma):
    n = len(beta_gamma)//2
    circuit = create_circuit(circuit_init, beta_gamma[:n], beta_gamma[n:])
    if isinstance(circuit, qiskit.circuit.quantumcircuit.QuantumCircuit):
        return np.real(Hc.eval("matrix", circuit, get_aer_backend('statevector_simulator'))[0])
    elif isinstance(circuit, pyquil.quil.Program):
        qvm = pyquil.api.QVMConnection(endpoint=fc.sync_endpoint, compiler_endpoint=fc.compiler_endpoint)
        return np.real(qvm.pauli_expectation(circuit, sum(Hc)))

**Exercise 4** (2 points). The $p$ parameter defines the number of steps in the Trotterization. The real question from here is how we optimize the $\beta_i$ and $\gamma_i$ parameters. If we can find a method that makes fewer evaluations to arrive at the same result, that is a win, since we have to execute fewer loops on the quantum computer. Try various methods for minimizing the evaluate function. We used L-BFGS-B before. Try another one and write the outcome in an object called `result`. You will see that the number of function evaluation (`nfev`) differs and so does the function value.

In [ ]:
### BEGIN SOLUTION
result = minimize(evaluate_circuit, np.concatenate([beta, gamma]), method='Nelder-Mead')
### END SOLUTION

In [ ]:
result2 = minimize(evaluate_circuit, np.concatenate([beta, gamma]), method='L-BFGS-B')
import scipy
assert isinstance(result, scipy.optimize.optimize.OptimizeResult)
assert result2.nfev != result.nfev
print("Function evaluations: %d versus %d" % (result2.nfev, result.nfev))
print("Function values: %f versus %f" % (result2.fun, result.fun))

If the circuit had an infinite capacity, you should obtain a minimum of $-1.5$ (minimum value of the Hamiltonian we defined above). However, with $p=2$, you might have a bigger value. Try increasing $p$ to see the effect on the minimum reached by the circuit.

Variational circuits are actually very similar to neural networks: we do some kind of gradient descent over a parameter space. Automatic differentiation of neural networks has been a major step in scaling them up and we can expect that similar techniques in variational quantum circuit can be useful. This is exactly what projects like [QuantumFlow](https://github.com/rigetti/quantumflow) and [PennyLane](https://pennylane.ai/) try.